In [1]:
import pandas as pd
from weighted_descriptors import compute_coupled_descriptors


In [2]:
import pandas as pd
from pathlib import Path
# 假设函数已经在当前文件中或者导入过
# from weighted_descriptors import compute_coupled_descriptors

# -------------------------------
# 1️⃣ 设置文件路径
# -------------------------------
file_path = r"data.xlsx"
output_path = r"example_output.xlsx"
df_result = compute_coupled_descriptors(file_path, output_path="example_output.xlsx")
# -------------------------------
# 2️⃣ 调用函数计算混合描述符
# -------------------------------
try:
    df_result = compute_coupled_descriptors(file_path, output_path=output_path)
    print("✅ 混合描述符计算完成！")
except ValueError as e:
    print(f"❌ 输入文件验证失败: {e}")
except Exception as e:
    print(f"❌ 计算过程中出错: {e}")

# -------------------------------
# 3️⃣ 查看结果
# -------------------------------
# 打印前五行
print(df_result.head())

# 如果没有提供 output_path，也可以手动保存
if output_path is None:
    df_result.to_excel("weighted_descriptors_result.xlsx", index=False)
    print("已手动保存结果到 weighted_descriptors_result.xlsx")



✅ 已保存加权混合描述符结果至：example_output.xlsx
❌ 计算过程中出错: [Errno 13] Permission denied: 'example_output.xlsx'
           species endpoint                   effect  Duration_Value  \
0  Vibrio fischeri     EC50  luminescence inhibition            0.25   
1  Vibrio fischeri     EC50  luminescence inhibition            0.25   
2  Vibrio fischeri     EC50  luminescence inhibition            0.25   
3  Vibrio fischeri     EC50  luminescence inhibition            0.25   
4  Vibrio fischeri     EC50  luminescence inhibition            0.25   

   ETA_Epsilon_2    GATS4e    GATS4c      TDB5s  naaaC  minHBint5  ...  \
0       0.856248  1.151021  0.973680  16.708067    0.0   0.000000  ...   
1       0.855721  1.164841  0.979460  16.491798    0.0   0.000000  ...   
2       0.858305  1.162362  0.981677  16.857363    0.0   0.019568  ...   
3       0.955326  1.898476  1.039049   0.166630    0.0   0.000000  ...   
4       0.955242  1.900696  1.039978   0.131878    0.0   0.000000  ...   

     AATSC0v  RPSA_mix2

# 2.预测

## 2.1 保存标准化参数

In [19]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib

import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib

def normalize_by_group(
    data_path, sheet_name, output_path=None, scaler_save_path="scalers_for_groups.save"
):
    """
    对指定活动表的数据按分组进行 Z-score 标准化，并保存标准化器。
    每个分组由 ("species", "endpoint", "effect", "Duration_Value") 唯一确定。

    :param data_path: 输入 Excel 文件路径
    :param sheet_name: 要处理的活动表名称
    :param output_path: 可选，保存标准化后的完整数据路径
    :param scaler_save_path: 保存每个分组 StandardScaler 的路径
    :return: 标准化后的 DataFrame
    """
    # 读取 Excel
    df = pd.read_excel(data_path, sheet_name=sheet_name)

    # 分组
    grouped = df.groupby(["species", "endpoint", "effect", "Duration_Value"])
    scaler_dict = {}
    normalized_list = []

    for name, group in grouped:
        # 特征列假设从第15列开始
        features = group.iloc[:, 14:]

        # Z-score 标准化
        scaler = StandardScaler()
        features_normalized = scaler.fit_transform(features)
        group.iloc[:, 14:] = features_normalized

        # 保存 scaler
        scaler_dict[name] = scaler

        normalized_list.append(group)

    # 合并所有分组
    df_normalized = pd.concat(normalized_list, axis=0)

    # 保存标准化后的完整数据（可选）
    if output_path:
        df_normalized.to_excel(output_path, index=False)
        print(f"✅ 标准化后的数据已保存到 {output_path}")

    # 保存所有分组的 StandardScaler
    joblib.dump(scaler_dict, scaler_save_path)
    print(f"✅ 每个分组的标准化器已保存到 {scaler_save_path}")

    return df_normalized

# -------------------------
# 使用示例
# -------------------------
data_path = "data01.xlsx"
sheet_name = "Combined Results"
output_path = "data01_normalized.xlsx"

df_norm = normalize_by_group(
    data_path=data_path,
    sheet_name=sheet_name,
    output_path=output_path,
    scaler_save_path="scalers_for_groups.save"
)


✅ 标准化后的数据已保存到 data01_normalized.xlsx
✅ 每个分组的标准化器已保存到 scalers_for_groups.save


## 2.2 模型定义

In [10]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from torch.optim import Adam
from tqdm import tqdm

class TransformerMLP(nn.Module):
    def __init__(self, input_dim, hidden_dim, transformer_layers, num_heads, ff_dim, output_dim):
        super(TransformerMLP, self).__init__()
        if input_dim % num_heads != 0:
            for i in range(num_heads, 0, -1):
                if input_dim % i == 0:
                    num_heads = i
                    break
            print(f"Adjusted num_heads to {num_heads} for compatibility with input_dim {input_dim}")
        self.encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim,
            nhead=num_heads,
            dim_feedforward=ff_dim,
            activation='relu'
        )
        self.transformer = nn.TransformerEncoder(self.encoder_layer, num_layers=transformer_layers)
        self.fc = nn.Sequential(
            nn.Linear(input_dim, hidden_dim),
            nn.ReLU(),
            nn.Linear(hidden_dim, output_dim)
        )

    def forward(self, x):
        x = x.unsqueeze(1)
        x = x.permute(1, 0, 2)
        x = self.transformer(x)
        x = x.permute(1, 0, 2)
        x = x.mean(dim=1)
        return self.fc(x)


def load_and_preprocess_data(file_path, train_sheet, test_sheet, target_column):
    # 读取训练集和测试集
    train_df = pd.read_excel(file_path, sheet_name=train_sheet)
    test_df = pd.read_excel(file_path, sheet_name=test_sheet)

    # 分离特征与目标
    X_train = train_df.drop(columns=[target_column])
    y_train = train_df[target_column]
    X_test = test_df.drop(columns=[target_column])
    y_test = test_df[target_column]

    # 合并再做 one-hot 编码（保持训练集和测试集特征一致）
    combined = pd.concat([X_train, X_test], axis=0)
    combined_encoded = pd.get_dummies(combined)

    # 拆分回训练集与测试集
    X_train_encoded = combined_encoded.iloc[:X_train.shape[0], :]
    X_test_encoded = combined_encoded.iloc[X_train.shape[0]:, :]

    # 🚫 不再使用 StandardScaler()
    # ✅ 因为 Excel 数据已经是 Z-score 标准化结果

    # 转换为 tensor
    return (
        torch.tensor(X_train_encoded.values, dtype=torch.float32),
        torch.tensor(y_train.values, dtype=torch.float32),
        torch.tensor(X_test_encoded.values, dtype=torch.float32),
        torch.tensor(y_test.values, dtype=torch.float32)
    )



def train_transformer_mlp_model(X_train, y_train, input_dim, epochs=100, batch_size=32, learning_rate=1e-3):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model = TransformerMLP(input_dim, hidden_dim=100, transformer_layers=5, num_heads=5, ff_dim=256, output_dim=1)
    model.to(device)
    criterion = nn.MSELoss()
    optimizer = Adam(model.parameters(), lr=learning_rate)
    dataset = TensorDataset(X_train, y_train.unsqueeze(1))
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    best_model = None
    best_score = -np.inf
    train_predictions = []
    model.train()
    for epoch in tqdm(range(epochs), desc="Training Epochs"):
        epoch_predictions = []
        for X_batch, y_batch in dataloader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()
            epoch_predictions.extend(y_pred.cpu().detach().numpy().flatten())
        r2 = r2_score(y_train.numpy(), epoch_predictions)
        if r2 > best_score:
            best_score = r2
            best_model = model.state_dict()
        train_predictions = epoch_predictions
    model.load_state_dict(best_model)
    return model, train_predictions

def evaluate_model(model, X, y, dataset_name, output_path):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()
    with torch.no_grad():
        predictions = model(X.to(device)).cpu().numpy().flatten()
    r2 = r2_score(y.numpy(), predictions)
    rmse = np.sqrt(mean_squared_error(y.numpy(), predictions))
    mae = mean_absolute_error(y.numpy(), predictions)
    print(f"{dataset_name} Set Evaluation:")
    print(f"  R2 Score: {r2:.4f}")
    print(f"  RMSE: {rmse:.4f}")
    print(f"  MAE: {mae:.4f}")
    results_df = pd.DataFrame({
        "Actual": y.numpy(),
        "Predicted": predictions
    })
    results_df.to_excel(output_path, index=False)
    return {"R2": r2, "RMSE": rmse, "MAE": mae}

In [13]:
import torch
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import pandas as pd
import joblib


import torch
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler  # 仍需导入，为保持兼容性
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

# 1. 读取训练集 Excel
file_path = r"F:\T-prediction\data_train.xlsx"  # 替换为你的训练集路径
target_column = "pEC50M (Mol/L)"  # 替换为你 Excel 中的目标列名
train_df = pd.read_excel(file_path)

# 2. 准备特征和目标
X_train = train_df.drop(columns=[target_column])
y_train = train_df[target_column]

# 仅对分类特征进行 one-hot 编码（不再标准化）
categorical_cols = ["species", "endpoint", "effect", "Duration_Value"]
X_train_encoded = pd.get_dummies(X_train, columns=categorical_cols)

# 🚫 不进行 Z 标准化，因为数据已经是标准化后的
# scaler = StandardScaler()
# X_train_encoded[X_train_encoded.columns] = scaler.fit_transform(X_train_encoded)

# 转换为 tensor
X_train_tensor = torch.tensor(X_train_encoded.values.astype(np.float32))
y_train_tensor = torch.tensor(y_train.values.astype(np.float32))


# 创建模型
input_dim = X_train_tensor.shape[1]
model = TransformerMLP(input_dim, hidden_dim=100, transformer_layers=5, num_heads=5, ff_dim=256, output_dim=1)

# 训练模型
model, train_predictions = train_transformer_mlp_model(X_train_tensor, y_train_tensor, input_dim)

# 保存模型权重
torch.save(model.state_dict(), "Transformer_DNN_model.pth")
print("✅ 模型权重已保存")

# --------------------------
# 保存 one-hot 编码列顺序
# --------------------------
feature_columns = X_train_encoded.columns.tolist()
joblib.dump(feature_columns, "feature_columns.save")
print("✅ 特征列顺序已保存")

# --------------------------
# 因为没有使用标准化器，所以不保存 scaler
# --------------------------
# joblib.dump(scaler, "scaler.save")

# --------------------------
# 训练集评估
# --------------------------
model.eval()
with torch.no_grad():
    y_pred = model(X_train_tensor).cpu().numpy().flatten()

r2 = r2_score(y_train_tensor.numpy(), y_pred)
rmse = mean_squared_error(y_train_tensor.numpy(), y_pred, squared=False)
mae = mean_absolute_error(y_train_tensor.numpy(), y_pred)

print(f"训练集评估结果:")
print(f"  R2: {r2:.4f}")
print(f"  RMSE: {rmse:.4f}")
print(f"  MAE: {mae:.4f}")

# 可选：保存训练集预测结果到 Excel
results_df = pd.DataFrame({"Actual": y_train_tensor.numpy(), "Predicted": y_pred})
results_df.to_excel("train_predictions.xlsx", index=False)
print("✅ 训练集预测结果已保存")



c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\torch\nn\modules\transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Adjusted num_heads to 1 for compatibility with input_dim 37
Adjusted num_heads to 1 for compatibility with input_dim 37


Training Epochs: 100%|██████████| 100/100 [00:50<00:00,  2.00it/s]

✅ 模型权重已保存
✅ 特征列顺序已保存
训练集评估结果:
  R2: 0.9679
  RMSE: 0.3533
  MAE: 0.2285
✅ 训练集预测结果已保存



c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [ ]:
import streamlit as st
import pandas as pd
import torch
import joblib
from transformer_model import TransformerMLP  # 替换为你保存模型类的文件名

st.set_page_config(page_title="T-prediction", layout="wide")
st.title("T-prediction: 预测工具（测试版）")

uploaded_file = st.file_uploader("上传待预测的 Excel 文件", type=["xlsx"])

if uploaded_file:
    try:
        st.info("正在读取文件并进行数据预处理...")
        predict_df = pd.read_excel(uploaded_file)

        # 读取特征列和标准化器
        feature_columns = joblib.load("feature_columns.save")
        scaler = joblib.load("scaler.save")

        # one-hot 编码
        categorical_cols = ["species", "endpoint", "effect", "Duration_Value"]
        predict_encoded = pd.get_dummies(predict_df, columns=categorical_cols)

        # 对齐列顺序
        for col in feature_columns:
            if col not in predict_encoded.columns:
                predict_encoded[col] = 0
        predict_encoded = predict_encoded[feature_columns]

        # 标准化
        predict_scaled = scaler.transform(predict_encoded)
        X_predict = torch.tensor(predict_scaled, dtype=torch.float32)

        # 加载模型
        input_dim = X_predict.shape[1]
        model = TransformerMLP(input_dim, hidden_dim=100, transformer_layers=5, num_heads=5, ff_dim=256, output_dim=1)
        model.load_state_dict(torch.load("Transformer_DNN_model.pth", map_location=torch.device('cpu')))
        model.eval()

        # 预测
        with torch.no_grad():
            predictions = model(X_predict).numpy().flatten()

        # 显示
        predict_df["Predicted"] = predictions
        st.success("预测完成！")
        st.dataframe(predict_df)

        # 下载
        output_file = "prediction_results.xlsx"
        predict_df.to_excel(output_file, index=False)
        st.download_button(
            label="下载预测结果",
            data=open(output_file, "rb"),
            file_name="prediction_results.xlsx"
        )

    except Exception as e:
        st.error(f"预测失败: {e}")
